Создание черного одноканального изображения 500*600.

In [1]:
import numpy as np
from PIL import Image

data = np.zeros((500, 600), dtype='|u1')
img = Image.fromarray(data)
img.save('pictures/colors/black.png')

Создание белого одноканального изображения 500*600.

In [2]:
import numpy as np
from PIL import Image

data = np.full((500, 600), 255, dtype='|u1')
img = Image.fromarray(data)
img.save('pictures/colors/white.png')

Создание красного RGB изображения 500*600.
Используется собственная обертка в `main.py`.

In [3]:
from core import CustomImage
from names import RED

img = CustomImage(600, 500)
img.fill(RED)
img.save('pictures/colors/red.png')

Создание градиентного RGB изображения 500*600.

In [4]:
from core import CustomImage

width = 600
height = 500
img = CustomImage(width, height)
for i in range(width):
    for j in range(height):
        val = (i + j) % 256
        img.set(i, j, (val, val, val))
img.save('pictures/colors/gradient.png')

Построение простейшей прямой (1 алгоритм)

In [5]:
from core import CustomImage, Color
from utils import float_range, star
from names import WHITE


def line(x0: int, y0: int, x1: int, y1: int, image: CustomImage, color: Color):
    for t in float_range(0, 1, 0.01, 8):
        x = x0 * (1.0 - t) + x1 * t
        y = y0 * (1.0 - t) + y1 * t
        image.set(round(x), round(y), color)


img = star(line, WHITE)
img.save('pictures/lines/line_1.png')

Второй вариант построения прямой (2 алгоритм)

In [6]:
from core import CustomImage, Color
from utils import star
from names import WHITE


def line(x0: int, y0: int, x1: int, y1: int, image: CustomImage, color: Color):
    for x in range(x0, x1, 1):
        t = (x - x0) / float(x1 - x0)
        y = y0 * (1.0 - t) + y1 * t
        image.set(round(x), round(y), color)


img = star(line, WHITE)
img.save('pictures/lines/line_2.png')

Третий вариант построения прямой (3 алгоритм)

In [7]:
from core import CustomImage, Color
from utils import star
from names import WHITE


def line(x0: int, y0: int, x1: int, y1: int, image: CustomImage, color: Color):
    steep = False
    if abs(x0 - x1) < abs(y0 - y1):
        x0, y0 = y0, x0
        x1, y1 = y1, x1
        steep = True
    if x0 > x1:
        x0, x1 = x1, x0
        y0, y1 = y1, y0
    for x in range(x0, x1, 1):
        t = (x - x0) / float(x1 - x0)
        y = y0 * (1. - t) + y1 * t
        if steep:
            image.set(round(y), round(x), color)
        else:
            image.set(round(x), round(y), color)


img = star(line, WHITE)
img.save('pictures/lines/line_3.png')

Четвертый вариант построения прямой (Алгоритм Брезенхема)

In [8]:
from core import CustomImage
from utils import star
from names import WHITE

def line(x0: int, y0: int, x1: int, y1: int, image: CustomImage, color: Color):
    image.line(x0, y0, x1, y1, color)

img = star(line, WHITE)
img.save('pictures/lines/line_4.png')

Отрисовка вершин трехмерной модели.

In [9]:
from core import CustomImage, ObjModel
from names import WHITE, FOX, DEER

fox_obj = ObjModel(FOX)
deer_obj = ObjModel(DEER)

transformations = [
    lambda x, y, z: (round(50 * x + 500), round(50 * y + 500)),
    lambda x, y, z: (round(100 * x + 500), round(100 * y + 500)),
    lambda x, y, z: (round(500 * x + 500), round(500 * y + 500)),
    lambda x, y, z: (round(4000 * x + 500), round(4000 * y + 500)),
]

for i, transform in enumerate(transformations):
    img = CustomImage(1000, 1000)
    fox_obj.draw_vertices(img, WHITE, transform)
    img.save(f'pictures/vertices/fox_{i + 1}.png')
for i, transform in enumerate(transformations):
    img = CustomImage(1000, 1000)
    deer_obj.draw_vertices(img, WHITE, transform)
    img.save(f'pictures/vertices/deer_{i + 1}.png')

Отрисовка ребер трехмерной модели.

In [10]:
from core import CustomImage, ObjModel
from names import WHITE, FOX, DEER

fox_obj = ObjModel(FOX)
deer_obj = ObjModel(DEER)

img = CustomImage(1000, 1000)
fox_obj.draw_faces(img, WHITE, lambda x, y, z: (4 * round(z) + 500, 4 * -round(y) + 500))
img.save('pictures/faces/fox.png')

img = CustomImage(1500, 1500)
deer_obj.draw_faces(img, WHITE, lambda x, y, z: (round(x) + 800, -round(y) + 1500))
img.save('pictures/faces/deer.png')